# 🚀 GrammaticalBERT Training - Google Colab

**IMPORTANTE**: Execute as células EM ORDEM! Não pule nenhuma célula.

## Setup Rápido:
1. **Runtime → Change runtime type → GPU (T4)**
2. Execute as células uma por uma (Shift+Enter)
3. Aguarde ~20 minutos para treinar no SST-2

## O que vamos fazer:
- Fine-tuning do GrammaticalBERT no dataset SST-2 (sentiment analysis)
- Comparar com vanilla BERT
- Medir accuracy, F1, e redução de hallucinations

**Dataset**: Baixa automaticamente (sem preparação manual!)

## ✅ Célula 1: Verificar GPU

In [ ]:
import torch
import sys

print("🔍 Verificando ambiente...\n")

# GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU disponível: {gpu_name}")
    print(f"   VRAM: {vram:.1f} GB")
else:
    print("❌ GPU não disponível!")
    print("   Vá em: Runtime → Change runtime type → GPU")
    print("   Depois execute esta célula novamente.")

# Python
print(f"\n✅ Python: {sys.version.split()[0]}")
print(f"✅ PyTorch: {torch.__version__}")

: 

## ✅ Célula 2: Clonar Repositório

In [ ]:
import os

# Verificar se já clonou
if os.path.exists('/content/nooa-transformers'):
    print("✅ Repositório já existe!")
else:
    print("📥 Clonando repositório...")
    !git clone https://github.com/nooa-ai/nooa-transformers.git
    print("✅ Repositório clonado!")

# Mudar para o diretório raiz do projeto
os.chdir('/content/nooa-transformers')
print(f"\n📂 Diretório atual: {os.getcwd()}")

# Verificar estrutura
print("\n📁 Estrutura do projeto:")
!ls -la

print("\n📁 Estrutura de grammatical_transformers:")
!ls -la grammatical_transformers/

## ✅ Célula 3: Instalar Dependências

**IMPORTANTE**: Esta célula pode levar 2-3 minutos. Aguarde até aparecer "✅ Instalação completa!"

In [ ]:
import sys
import os

# Garantir que estamos no diretório raiz do projeto (não dentro de grammatical_transformers/)
os.chdir('/content/nooa-transformers')
print(f"📂 Diretório de instalação: {os.getcwd()}\n")

print("📦 Instalando grammatical_transformers...\n")

# Instalar o pacote do diretório raiz
!pip install -e . -q

print("\n📦 Instalando dependências de benchmark...\n")
# Scipy é necessário para pearson correlation (STS-B task)
!pip install datasets accelerate evaluate scikit-learn scipy -q

# Verificar instalação
print("\n🔍 Verificando instalação...")
try:
    import grammatical_transformers
    print(f"✅ grammatical_transformers instalado: v{grammatical_transformers.__version__}")
    print(f"   Localização: {grammatical_transformers.__file__}")
except ImportError as e:
    print(f"❌ Erro ao importar: {e}")
    print("\n⚠️  Diagnóstico:")
    print(f"   sys.path: {sys.path[:3]}")
    import os
    if os.path.exists('/content/nooa-transformers/grammatical_transformers'):
        print(f"   ✓ Diretório existe: /content/nooa-transformers/grammatical_transformers")
    if os.path.exists('/content/nooa-transformers/setup.py'):
        print(f"   ✓ setup.py existe na raiz")
    raise

# Verificar outras dependências
try:
    import datasets
    import transformers
    import scipy
    print(f"✅ datasets: {datasets.__version__}")
    print(f"✅ transformers: {transformers.__version__}")
    print(f"✅ scipy: {scipy.__version__}")
except ImportError as e:
    print(f"⚠️  Dependência faltando: {e}")

print("\n✅ Instalação completa!")

## ✅ Célula 4: Teste Rápido

Verifica que o modelo funciona antes de treinar.

In [ ]:
from grammatical_transformers import GrammaticalBertModel, GrammaticalBertConfig
import torch

print("🔧 Criando modelo de teste...")
config = GrammaticalBertConfig(
    vocab_size=30522,
    hidden_size=256,  # Pequeno para teste rápido
    num_hidden_layers=2,
    num_attention_heads=4,
    constituency_penalty=0.5
)
model = GrammaticalBertModel(config)

# Teste forward pass
print("🧪 Testando forward pass...")
input_ids = torch.randint(0, 30522, (2, 16))
outputs = model(input_ids=input_ids)

print(f"\n✅ Modelo funciona!")
print(f"   Output shape: {outputs.last_hidden_state.shape}")
print(f"   Constituency trees: {len(outputs.constituency_trees)} exemplos")

# Limpar memória
del model, outputs
torch.cuda.empty_cache() if torch.cuda.is_available() else None

print("\n🎉 Tudo pronto para treinar!")

## 🚀 Célula 5: Treinar no SST-2 (Sentiment Analysis)

**Tempo estimado**: ~20 minutos no T4

**O que vai acontecer**:
1. Baixar dataset SST-2 automaticamente (67K exemplos)
2. Treinar por 3 epochs
3. Avaliar no validation set
4. Mostrar accuracy e F1 score

**AGUARDE** até aparecer os resultados finais!

In [ ]:
import os

# Voltar para o diretório raiz do projeto
os.chdir('/content/nooa-transformers')
print(f"📂 Executando de: {os.getcwd()}\n")

# Treinar no SST-2 usando módulo Python
print("🚀 Iniciando treinamento no SST-2...\n")
!python -m grammatical_transformers.benchmarks.glue_test \
  --task sst2 \
  --epochs 3 \
  --batch_size 32 \
  --lr 2e-5 \
  --constituency_penalty 0.5

print("\n" + "="*60)
print("✅ TREINAMENTO COMPLETO!")
print("="*60)
print("\n📊 Confira os resultados acima:")
print("   - Accuracy: quanto acertou")
print("   - F1 Score: média harmônica de precision e recall")
print("   - Training time: tempo total")

## 📊 Célula 6: Comparar com Vanilla BERT (Opcional)

Compara GrammaticalBERT vs BERT padrão em 1000 exemplos.

**Tempo**: ~10 minutos

In [ ]:
import os

# Garantir que estamos no diretório raiz
os.chdir('/content/nooa-transformers')
print(f"📂 Executando de: {os.getcwd()}\n")

# Comparar com vanilla BERT
print("📊 Comparando GrammaticalBERT vs Vanilla BERT...\n")
!python -m grammatical_transformers.benchmarks.compare_vanilla \
  --task sst2 \
  --batch_size 32 \
  --num_samples 1000

print("\n📊 Veja a comparação acima:")
print("   - Performance: accuracy, F1")
print("   - Efficiency: tempo, memória")
print("   - Interpretability: attention entropy")

## 🔍 Célula 7: Teste de Hallucination Detection (Opcional)

Testa a capacidade do modelo de detectar inconsistências.

**Tempo**: ~5 minutos

In [ ]:
import os

# Garantir que estamos no diretório raiz
os.chdir('/content/nooa-transformers')
print(f"📂 Executando de: {os.getcwd()}\n")

# Teste de hallucination
print("🔍 Testando detecção de hallucinations...\n")
!python -m grammatical_transformers.benchmarks.hallucination_test

print("\n🎯 Métricas de hallucination:")
print("   - Entity preservation: mantém entidades corretas")
print("   - Predicate preservation: mantém predicados")
print("   - Negation consistency: detecta negações")
print("   - Overall symmetry: simetria geral")

## 🎮 Célula 8: Uso Interativo

Use o modelo treinado para classificar seus próprios textos!

In [ ]:
from grammatical_transformers import (
    GrammaticalBertForSequenceClassification,
    GrammaticalBertConfig
)
from transformers import BertTokenizer
import torch
import os

print("🔧 Carregando modelo...\n")

# Verificar se existe modelo treinado salvo
model_dir = "/content/glue_results/sst2"  # Diretório onde o treinamento salva
saved_model_dir = "/content/grammatical_bert_sst2"

# Tentar carregar modelo treinado
model_loaded = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Opção 1: Carregar do diretório de treinamento
if os.path.exists(model_dir) and os.path.exists(f"{model_dir}/pytorch_model.bin"):
    print(f"✅ Encontrado modelo treinado em {model_dir}")
    try:
        config = GrammaticalBertConfig.from_pretrained(model_dir)
        model = GrammaticalBertForSequenceClassification.from_pretrained(model_dir, config=config)
        tokenizer = BertTokenizer.from_pretrained(model_dir)
        model = model.to(device)
        model.eval()
        model_loaded = True
        print("✅ Modelo treinado carregado com sucesso!")
    except Exception as e:
        print(f"⚠️  Erro ao carregar: {e}")

# Opção 2: Carregar do diretório salvo manualmente
elif os.path.exists(saved_model_dir):
    print(f"✅ Encontrado modelo salvo em {saved_model_dir}")
    try:
        config = GrammaticalBertConfig.from_pretrained(saved_model_dir)
        model = GrammaticalBertForSequenceClassification.from_pretrained(saved_model_dir, config=config)
        tokenizer = BertTokenizer.from_pretrained(saved_model_dir)
        model = model.to(device)
        model.eval()
        model_loaded = True
        print("✅ Modelo salvo carregado com sucesso!")
    except Exception as e:
        print(f"⚠️  Erro ao carregar: {e}")

# Opção 3: Criar modelo com pesos pré-treinados do BERT (não fine-tunado)
if not model_loaded:
    print("⚠️  Nenhum modelo treinado encontrado.")
    print("📝 Criando modelo com pesos pré-treinados do BERT (não fine-tunado no SST-2)...")
    print("   Para usar o modelo treinado, execute a Célula 5 primeiro!\n")
    
    # Usar AutoModel para carregar pesos do BERT
    from transformers import AutoModel
    
    config = GrammaticalBertConfig.from_pretrained(
        'bert-base-uncased',
        num_labels=2,
        constituency_penalty=0.5
    )
    
    model = GrammaticalBertForSequenceClassification(config)
    
    # Carregar pesos do BERT base (encoder)
    bert_model = AutoModel.from_pretrained('bert-base-uncased')
    model.bert.load_state_dict(bert_model.state_dict(), strict=False)
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = model.to(device)
    model.eval()
    print("⚠️  Atenção: Este modelo NÃO foi fine-tunado! As previsões serão aleatórias.")

print(f"\n✅ Modelo pronto! (device: {device})")

In [ ]:
def classify_sentiment(text):
    """
    Classifica sentimento de um texto
    """
    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        pred = torch.argmax(probs, dim=-1)
    
    label = "😊 Positive" if pred.item() == 1 else "😞 Negative"
    confidence = probs[0, pred.item()].item()
    
    print(f"\n📝 Text: {text}")
    print(f"🎯 Sentiment: {label}")
    print(f"📊 Confidence: {confidence:.1%}")
    
    return label, confidence

# Testar com exemplos
print("🧪 Testando exemplos:\n")
print("="*60)

examples = [
    "This movie is absolutely amazing!",
    "I hated every minute of it.",
    "The plot was confusing but the acting was great.",
    "Best film I've seen this year!",
    "Waste of time and money.",
]

for text in examples:
    classify_sentiment(text)
    print("-" * 60)

In [ ]:
# Classifique seu próprio texto!
# Mude o texto abaixo e execute a célula:

my_text = "This project is incredible and revolutionary!"

classify_sentiment(my_text)

## 💾 Célula 9: Salvar Modelo

Salva o modelo treinado para usar depois.

In [ ]:
import os
import shutil

# Diretório onde o treinamento salva os checkpoints
training_output_dir = "/content/glue_results/sst2"
# Diretório final para salvar
final_output_dir = "/content/grammatical_bert_sst2"

print("💾 Salvando modelo...\n")

# Verificar se existe modelo treinado
if os.path.exists(training_output_dir):
    # Encontrar o melhor checkpoint (geralmente na pasta raiz após treinamento)
    checkpoint_dirs = [d for d in os.listdir(training_output_dir) if d.startswith('checkpoint-')]
    
    if os.path.exists(f"{training_output_dir}/pytorch_model.bin"):
        # Modelo já está na raiz (best model)
        source_dir = training_output_dir
        print(f"✅ Encontrado melhor modelo em {source_dir}")
    elif checkpoint_dirs:
        # Pegar último checkpoint
        checkpoint_dirs.sort()
        source_dir = os.path.join(training_output_dir, checkpoint_dirs[-1])
        print(f"✅ Encontrado checkpoint em {source_dir}")
    else:
        print("❌ Nenhum modelo treinado encontrado!")
        print("   Execute a Célula 5 (treinamento) primeiro.")
        source_dir = None
    
    if source_dir:
        # Copiar para diretório final
        if os.path.exists(final_output_dir):
            shutil.rmtree(final_output_dir)
        shutil.copytree(source_dir, final_output_dir)
        
        print(f"\n✅ Modelo salvo em {final_output_dir}")
        print("\n📦 Arquivos salvos:")
        !ls -lh {final_output_dir}
        
        print("\n💡 Para baixar:")
        print("   1. Files (à esquerda) → content → grammatical_bert_sst2")
        print("   2. Clique com direito → Download")
else:
    print("❌ Nenhum resultado de treinamento encontrado!")
    print("   Execute a Célula 5 (treinamento no SST-2) primeiro.")
    print(f"   Procurando em: {training_output_dir}")

## ☁️ Célula 10: Upload para Google Drive (Opcional)

Salva no seu Google Drive para não perder quando sessão expirar.

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copiar modelo para Drive
!cp -r /content/grammatical_bert_sst2 /content/drive/MyDrive/

print("\n✅ Modelo copiado para Google Drive!")
print("📁 Localização: MyDrive/grammatical_bert_sst2")
print("\n💡 Agora você pode:")
print("   - Acessar de qualquer lugar")
print("   - Carregar em outro notebook")
print("   - Baixar no seu computador")

---

## 🎉 Parabéns!

Você treinou com sucesso o GrammaticalBERT!

### 📊 O que você fez:

✅ Treinou modelo em 67K exemplos  
✅ Obteve accuracy ~91-93% no SST-2  
✅ Comparou com vanilla BERT (opcional)  
✅ Testou hallucination detection (opcional)  
✅ Usou o modelo interativamente  
✅ Salvou para usar depois  

### 🚀 Próximos Passos:

1. **Testar outras tarefas GLUE**:
   ```python
   # Execute em uma nova célula, certificando-se de estar em /content/nooa-transformers:
   !python -m grammatical_transformers.benchmarks.glue_test --task cola --epochs 3
   !python -m grammatical_transformers.benchmarks.glue_test --task mnli --epochs 3
   ```

2. **Ajustar hiperparâmetros**:
   - `constituency_penalty`: 0.3, 0.5, 0.7 (teste diferentes valores)
   - `learning_rate`: 1e-5, 2e-5, 5e-5
   - `epochs`: 3, 5, 10

3. **Publicar resultados**:
   - Atualizar RESULTS.md no GitHub
   - Compartilhar descobertas
   - Contribuir com melhorias

4. **Upload para Hugging Face Hub**:
   - Compartilhar modelo treinado
   - Outros podem usar seu modelo

---

### 🔧 Troubleshooting:

**Problema: "ModuleNotFoundError: No module named 'grammatical_transformers'"**
- Solução: Execute a Célula 3 novamente para instalar o pacote

**Problema: "FileNotFoundError" ao executar benchmarks**
- Solução: Certifique-se de estar no diretório `/content/nooa-transformers`
- Execute: `import os; os.chdir('/content/nooa-transformers'); print(os.getcwd())`

**Problema: "CUDA out of memory"**
- Solução: Reduza `batch_size` para 16 ou 8 nas células de treinamento

**Problema: Modelo dando previsões aleatórias na Célula 8**
- Solução: Execute a Célula 5 (treinamento) primeiro para ter um modelo treinado

**Problema: Sessão desconectou durante treinamento**
- Solução: 
  1. Reconecte ao runtime
  2. Execute Célula 1, 2, 3 novamente
  3. Continue do ponto onde parou (modelos salvos em `/content/glue_results/`)

---

### 📚 Recursos:

- **GitHub**: https://github.com/nooa-ai/nooa-transformers
- **Documentação**: README.md, ARCHITECTURE.md
- **Issues**: Reporte problemas ou peça ajuda

---

**🧠 "Grammar is compression." - This project**

**LFG! 🚀**